# CSIRO Biomass – Inference Notebook

This notebook blends precomputed pillar submissions (SigLIP, DINO, MVP, Dinov2) into the final `submission.csv`.

**Usage**
1. Attach the dataset that contains the four pillar CSVs.
2. Update the file paths in the next cell if your dataset uses different names.
3. Run all cells – no training is performed here, so scoring should complete quickly.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

# === Configure pillar submission paths ===
SIGLIP_PATH = Path('/kaggle/input/csiro-pillars-run5/submission_siglip.csv')
DINO_PATH   = Path('/kaggle/input/csiro-pillars-run5/submission_dino.csv')
MVP_PATH    = Path('/kaggle/input/csiro-pillars-run5/submission_mvp.csv')
DINOV2_PATH = Path('/kaggle/input/csiro-pillars-run5/submission_dinov2.csv')
SAMPLE_SUB_PATH = Path('/kaggle/input/csiro-biomass/sample_submission.csv')

# Default SigLIP-heavy weights (SigLIP / DINO / MVP / Dinov2)
WEIGHTS = np.array([0.60, 0.20, 0.10, 0.10], dtype=np.float64)

# Optional clipping of target predictions
CLIP_MIN = 0.0
CLIP_MAX = None

print('Configured weights (unnormalized):', WEIGHTS)


In [ ]:
SAMPLE_COLS = ('sample_id', 'target')

def load_submission(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f'Missing submission file: {path}')
    df = pd.read_csv(path)
    missing = [c for c in SAMPLE_COLS if c not in df.columns]
    if missing:
        raise ValueError(f"{path} missing column(s): {missing}")
    return df[list(SAMPLE_COLS)].copy()

weights = WEIGHTS.astype(np.float64)
if np.any(weights < 0):
    raise ValueError('Weights must be non-negative.')
if weights.sum() <= 0:
    raise ValueError('Weights sum to zero; provide positive weights.')
weights /= weights.sum()
print('Normalized weights:', weights.round(4))

siglip_df = load_submission(SIGLIP_PATH)
dino_df = load_submission(DINO_PATH)
mvp_df = load_submission(MVP_PATH)
dinov2_df = load_submission(DINOV2_PATH)

sample_template = pd.read_csv(SAMPLE_SUB_PATH)[['sample_id']].copy()
if sample_template['sample_id'].duplicated().any():
    raise ValueError('sample_submission contains duplicate sample_ids.')

def align_with_template(pillar_df: pd.DataFrame, name: str) -> np.ndarray:
    merged = sample_template.merge(pillar_df, on='sample_id', how='left', validate='one_to_one')
    if merged['target'].isna().any():
        missing = merged.loc[merged['target'].isna(), 'sample_id'].head().tolist()
        raise ValueError(f"{name} submission missing predictions for ids: {missing}")
    return merged['target'].to_numpy(dtype=np.float64)

stacked = np.stack([
    align_with_template(siglip_df, 'SigLIP'),
    align_with_template(dino_df, 'DINO'),
    align_with_template(mvp_df, 'MVP'),
    align_with_template(dinov2_df, 'Dinov2'),
], axis=0)

blended = np.tensordot(weights, stacked, axes=(0, 0))
if CLIP_MIN is not None or CLIP_MAX is not None:
    blended = np.clip(blended, CLIP_MIN, CLIP_MAX)

submission = sample_template.copy()
submission['target'] = blended
submission.to_csv('submission.csv', index=False)
print(f'Saved blended submission with {len(submission):,} rows to submission.csv')
submission.head()
